In [5]:
import pandas as pd
import numpy as np
from model import * 


### exchange rate fluctiations affecting EPC contract price 

In [6]:
# TODO add to inputs! 

current_exchange = 1.30 # currently = 1.30 eur => us 

# price has fluctuated from 0.95 to 1.39 over past 10 years 
# https://www.xe.com/currencycharts/?from=EUR&to=USD&view=10Y
# lets do the full range, and then some, +/- 0.15
rates = np.round(np.arange(0.80, 1.55, 0.05),2)

# 0.01 increase in exchange rate => EPC contract price increases / decreases by 1,670,000
epc_fluct = 1670e3/0.01

devs = np.round(rates - current_exchange,3)
dev_cash = np.round(devs * epc_fluct,3)

print(rates, devs, dev_cash)

[0.8  0.85 0.9  0.95 1.   1.05 1.1  1.15 1.2  1.25 1.3  1.35 1.4  1.45
 1.5 ] [-0.5  -0.45 -0.4  -0.35 -0.3  -0.25 -0.2  -0.15 -0.1  -0.05  0.    0.05
  0.1   0.15  0.2 ] [-83500000. -75150000. -66800000. -58450000. -50100000. -41750000.
 -33400000. -25050000. -16700000.  -8350000.         0.   8350000.
  16700000.  25050000.  33400000.]


In [21]:
roe_dict = {}
for cash_delta, dev,rate in zip(dev_cash, devs, rates):
    b = FinancialModel()
    curr_epc_val = b.epc_data["EPC Cost"]
    
    new_val = curr_epc_val +cash_delta
    
    roe_dict[dev] = (b.eval_epc_currency_sens(new_val), cash_delta, rate)

roe_dict

{-0.5: (4.511611377349783, -83500000.0, 13.511611377349784),
 -0.45: (4.476191178753013, -75150000.0, 13.476191178753012),
 -0.4: (4.440770980156244, -66800000.0, 13.440770980156245),
 -0.35: (4.405350781559475, -58450000.0, 13.405350781559475),
 -0.3: (4.369930582962706, -50100000.0, 13.369930582962706),
 -0.25: (4.334510384365936, -41750000.0, 13.334510384365936),
 -0.2: (4.299090185769168, -33400000.0, 13.299090185769167),
 -0.15: (4.263669987172398, -25050000.0, 13.263669987172399),
 -0.1: (4.22824978857563, -16700000.0, 13.22824978857563),
 -0.05: (4.19282958997886, -8350000.0, 13.19282958997886),
 0.0: (4.157409391382091, 0.0, 13.15740939138209),
 0.05: (4.121989192785322, 8350000.0, 13.121989192785321),
 0.1: (4.086568994188553, 16700000.0, 13.086568994188553),
 0.15: (4.051148795591783, 25050000.0, 13.051148795591782),
 0.2: (4.0157285969950145, 33400000.0, 13.015728596995014)}

In [22]:
rates = [v[0] for v in roe_dict.values()]
rates[10]
delta_rates = [r - rates[10] for r in rates]
delta_rates


[0.35420198596769215,
 0.31878178737092266,
 0.2833615887741532,
 0.2479413901773846,
 0.2125211915806151,
 0.17710099298384563,
 0.14168079438707704,
 0.10626059579030755,
 0.07084039719353896,
 0.03542019859676948,
 0.0,
 -0.03542019859676859,
 -0.07084039719353807,
 -0.10626059579030755,
 -0.14168079438707615]

In [23]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [24]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=list(roe_dict.keys()),
    y=delta_rates, 
    mode='lines+markers',
    showlegend=False,
))

fig.update_layout(xaxis_title='Change in Interest Rate',
                  yaxis_title='Change in ROE',)

In [25]:
curr_epc_val

455000000.0